## Import

In [ ]:
import pandas as pd

import os 

import re

import json

from pathlib import Path

import random

from dotenv import load_dotenv

import google.generativeai as genai

from google.generativeai.types import HarmCategory, HarmBlockThreshold 







load_dotenv()

## def 

#### Persona Prompt

In [ ]:
# --- Hilfsfunktionen (unverändert) ---



def load_history_tweets_for_user(user_id_str: str, jsonl_dir: Path) -> list:

    """

    Lädt die History-Tweets für einen gegebenen Nutzer aus seiner JSONL-Datei.

    Gibt eine Liste von Tweet-Texten zurück.

    """

    user_file = jsonl_dir / f"{user_id_str}.jsonl"

    history_tweet_texts = []



    if not user_file.exists():

        print(f"FEHLER: Datendatei für Nutzer {user_id_str} nicht gefunden unter {user_file}")

        return history_tweet_texts



    try:

        with open(user_file, 'r', encoding='utf-8') as f:

            for line_number, line in enumerate(f):

                if line.strip() == "":

                    continue 

                try:

                    data = json.loads(line)

                    if data.get("set") == "history":

                        tweets = data.get("tweets", [])

                        for tweet in tweets:

                            if 'full_text' in tweet and tweet['full_text']:

                                history_tweet_texts.append(tweet['full_text'])

                        break 

                except json.JSONDecodeError as e:

                    print(f"WARNUNG: JSON-Dekodierungsfehler in Zeile {line_number + 1} der Datei {user_file}: {e}")

                    continue

        

        if not history_tweet_texts:

            print(f"WARNUNG: Keine History-Tweets für Nutzer {user_id_str} im 'history'-Set gefunden oder 'full_text' fehlt.")



    except Exception as e:

        print(f"FEHLER beim Lesen der Datei {user_file}: {e}")

    

    return history_tweet_texts



def format_tweets_for_llm(tweet_texts: list, max_tweets: int = 50, max_chars: int = 10000) -> str:

    """

    Formatiert eine Liste von Tweet-Texten zu einem einzigen String für den LLM-Input.

    Begrenzt die Anzahl der Tweets und die Gesamtzeichenzahl, um Kontextfenster nicht zu sprengen.

    """

    selected_tweets = tweet_texts[:max_tweets]

    

    formatted_string = ""

    for tweet in selected_tweets:

        if len(formatted_string) + len(tweet) + len("\n---\n") > max_chars:

            break

        formatted_string += tweet + "\n---\n"

        

    if not formatted_string and tweet_texts: 

        print(f"WARNUNG: Konnte Tweets nicht formatieren. Möglicherweise sind einzelne Tweets zu lang oder max_chars zu klein.")

        if tweet_texts[0] and len(tweet_texts[0]) <= max_chars:

            return tweet_texts[0]

        return ""



    return formatted_string.strip().rstrip("-")



def get_meta_prompt_for_persona_generation_json(num_candidates: int) -> str:

    """

    Gibt den Meta-Prompt für das LLM zur Generierung von Persona-Kandidaten

    im JSON-Format zurück.

    """

    meta_prompt = f"""

Sie sind ein KI-Assistent, der detaillierte Persona-Beschreibungen erstellt.

Basierend auf den folgenden Tweets eines Nutzers, analysieren Sie dessen Schreibstil, typische Themen, Tonalität und Interaktionsmuster.



Generieren Sie {num_candidates} unterschiedliche Persona-Beschreibungen.

Jede Beschreibung sollte ein prägnanter Absatz sein, der, wenn er einem Sprachmodell gegeben wird, dieses befähigen würde, den Twitter-Stil des Nutzers zu imitieren.



Konzentrieren Sie sich bei Ihrer Analyse und in den Persona-Beschreibungen auf diese Aspekte:

1.  **Schreibstil:** Satzstruktur, Vokabular (formell/informell, Umgangssprache, Fachbegriffe), Verwendung von Interpunktion, Groß-/Kleinschreibung, Emojis, Hashtags, Links.

2.  **Häufige Themen:** Über welche Themen twittert der Nutzer häufig? Was sind wiederkehrende Interessen oder Meinungen?

3.  **Tonalität:** Ist der Nutzer im Allgemeinen positiv, negativ, neutral, sarkastisch, humorvoll, analytisch, kritisch, unterstützend etc.?

4.  **Interaktionsmuster:** Wie interagiert der Nutzer in Antworten (falls Beispiele vorhanden sind)? Ist er gesprächig, argumentativ, unterstützend? Initiiert er oft Gespräche oder antwortet er hauptsächlich?



ANWEISUNGEN FÜR DAS AUSGABEFORMAT:

Ihre Antwort MUSS ein valides JSON-Objekt sein und sonst nichts.

Das JSON-Objekt soll einen einzigen Schlüssel namens "persona_descriptions" enthalten.

Der Wert dieses Schlüssels soll eine JSON-Liste von Strings sein. Jeder String in dieser Liste ist eine der {num_candidates} generierten Persona-Beschreibungen.

Fügen Sie keine einleitenden Sätze, Erklärungen oder Text außerhalb des JSON-Objekts hinzu.



Beispiel für das erwartete JSON-Format (bei num_candidates = 2):

{{

  "persona_descriptions": [

    "Dies ist die erste Persona-Beschreibung, die den Schreibstil, die Themen und die Tonalität des Nutzers X zusammenfasst...",

    "Die zweite Persona-Beschreibung für Nutzer X könnte sich auf andere Aspekte konzentrieren oder eine alternative Interpretation bieten..."

  ]

}}



Hier sind die Tweets des Nutzers:

"""

    return meta_prompt









def call_gemini_api_for_persona_gen(full_prompt_for_llm: str, num_candidates: int) -> list:

    """

    Ruft die Google Gemini API auf, um Persona-Prompt-Kandidaten als JSON zu generieren.

    Gibt eine Liste von Strings zurück, wobei jeder String "Persona X: Beschreibung..." ist.

    """

    print("\n--- SENDE PROMPT AN GOOGLE GEMINI API (JSON Modus) ---")

    # print(full_prompt_for_llm) # Optional für Debugging

    print("--- WARTE AUF ANTWORT VON GEMINI API ... ---")



    try:

        # Stellen Sie sicher, dass Ihr API-Schlüssel konfiguriert ist, z.B.:

        # api_key = os.getenv("GOOGLE_API_KEY") or os.getenv("GEMINI_API_KEY")

        # if not api_key:

        #     raise ValueError("API Key nicht in Umgebungsvariablen gefunden.")

        # genai.configure(api_key=api_key) # Idealerweise einmal zu Beginn Ihres Skripts/Notebooks



        model = genai.GenerativeModel(model_name='gemini-1.5-flash-latest') # oder Ihr bevorzugtes Modell



        generation_config = genai.types.GenerationConfig(

            candidate_count=1,

            max_output_tokens=20, # Passen Sie dies bei Bedarf an

            temperature=0.7,

            response_mime_type="application/json"  # WICHTIG: JSON-Ausgabe anfordern

        )



        safety_settings = {

            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,

            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,

            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,

            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,

        }



        response = model.generate_content(

            contents=full_prompt_for_llm,

            generation_config=generation_config,

            safety_settings=safety_settings

        )



        print("--- ANTWORT VON GEMINI API ERHALTEN (JSON Modus) ---")



        if not response.text: # type: ignore

            print("WARNUNG: Kein Text von der Gemini API erhalten (JSON Modus).")

            if response.prompt_feedback and response.prompt_feedback.block_reason: # type: ignore

                print(f"Blockierungsgrund: {response.prompt_feedback.block_reason}") # type: ignore

            return []

        

        # Für Debugging kann es hilfreich sein, den Rohtext auszugeben

        # print(f"--- ROH-ANTWORT VON GEMINI (sollte JSON sein) ---\n{response.text}")



        try:

            parsed_json = json.loads(response.text) # type: ignore

            raw_descriptions = parsed_json.get("persona_descriptions")



            if not isinstance(raw_descriptions, list):

                print(f"WARNUNG: JSON-Struktur unerwartet. 'persona_descriptions' ist keine Liste oder fehlt. Erhalten: {parsed_json}")

                # Fallback: Versuchen, den Rohtext als einzelne Persona zu verwenden, falls möglich

                if isinstance(parsed_json, str): # Manchmal gibt das Modell trotz allem nur einen String zurück

                     return [f"Persona 1: {parsed_json.strip()}"]

                return []



            # Validieren, dass die Liste Strings enthält und leere Strings filtern

            persona_candidates_from_json = [str(desc).strip() for desc in raw_descriptions if isinstance(desc, str) and str(desc).strip()]

            

            if not persona_candidates_from_json and raw_descriptions:

                 print(f"WARNUNG: 'persona_descriptions' enthielt Elemente, aber keine validen Strings nach Filterung. Original: {raw_descriptions}")

                 return []

            

            if not persona_candidates_from_json:

                print("WARNUNG: Keine Persona-Beschreibungen im JSON gefunden.")

                return []



            # Formatieren der Beschreibungen zu "Persona X: ..." für Konsistenz mit dem Rest des Codes

            # (falls das LLM dies nicht bereits im String tut)

            formatted_candidates = []

            for i, desc in enumerate(persona_candidates_from_json[:num_candidates]): # Nur die angeforderte Anzahl

                # Entferne ein eventuell vom LLM hinzugefügtes "Persona X:" Muster, um Doppelung zu vermeiden

                cleaned_desc = re.sub(r"^(Persona\s*\d*\s*:)\s*", "", desc, flags=re.IGNORECASE).strip()

                if cleaned_desc: # Nur hinzufügen, wenn nach der Bereinigung Text übrig ist

                    formatted_candidates.append(f"Persona {i + 1}: {cleaned_desc}")

            

            if not formatted_candidates and persona_candidates_from_json:

                 print("WARNUNG: Nach der Formatierung waren keine Kandidaten mehr übrig. Verwende Roh-JSON-Strings.")

                 # Fallback auf die ursprünglichen JSON-Strings mit "Persona X:" Präfix

                 return [f"Persona {i+1}: {desc_orig}" for i, desc_orig in enumerate(persona_candidates_from_json[:num_candidates])]





            print(f"--- {len(formatted_candidates)} Persona-Kandidaten erfolgreich aus JSON geparst und formatiert ---")

            return formatted_candidates



        except json.JSONDecodeError as e:

            print(f"FEHLER: Konnte JSON-Antwort von Gemini nicht parsen: {e}")

            print(f"Rohtext, der zum Fehler führte:\n{response.text}") # type: ignore

            # Als Fallback könnten Sie hier die alte, zeilenbasierte Parsing-Logik auf response.text anwenden,

            # aber das Ziel ist ja, diese zu vermeiden. Fürs Erste geben wir einen Fehler zurück.

            return []



    except Exception as e:

        print(f"FEHLER bei der Kommunikation mit der Gemini API (JSON Modus): {e}")

        return []

        



    



def generate_persona_candidates_for_user(user_id_str: str, jsonl_dir: Path, num_candidates: int) -> list:

    """

    Orchestriert das Laden von Tweets, die Aufbereitung und den LLM-Aufruf

    zur Generierung von Persona-Prompt-Kandidaten für einen Nutzer.

    """

    print(f"Starte Generierung von Persona-Prompt-Kandidaten für Nutzer: {user_id_str}")

    

    history_tweets = load_history_tweets_for_user(user_id_str, jsonl_dir)

    if not history_tweets:

        print(f"Konnte keine History-Tweets für {user_id_str} laden. Überspringe Persona-Generierung.")

        return []



    formatted_history_for_llm = format_tweets_for_llm(history_tweets)

    if not formatted_history_for_llm:

        print(f"Konnte History-Tweets für {user_id_str} nicht für LLM formatieren. Überspringe Persona-Generierung.")

        return []



    meta_prompt = get_meta_prompt_for_persona_generation_json(num_candidates)

    full_prompt_for_llm = meta_prompt + "\n\n" + formatted_history_for_llm



    persona_candidates = call_gemini_api_for_persona_gen(full_prompt_for_llm, num_candidates)

    

    print(f"Generierung für Nutzer {user_id_str} abgeschlossen. {len(persona_candidates)} Kandidaten erhalten.")

    return persona_candidates





def run_persona_prompt_generation(

    processing_mode: int,

    user_data_dir: Path,

    num_persona_candidates: int,

    results_output_dir: Path,

    specific_user_id_to_process: str | None = None, # type: ignore

    eligible_users_file: Path | None = None # type: ignore

):

    """

    Hauptfunktion zur Generierung von Persona-Prompts für Twitter-Nutzer.



    Args:

        processing_mode (int): 1 für spezifischen Nutzer, 2 für Nutzer aus Datei, 3 für alle im Verzeichnis.

        user_data_dir (Path): Verzeichnis mit den JSONL-Dateien der Nutzer.

        num_persona_candidates (int): Anzahl der zu generierenden Persona-Kandidaten pro Nutzer.

        results_output_dir (Path): Verzeichnis zum Speichern der Ergebnisse.

        specific_user_id_to_process (str, optional): Die ID des spezifischen Nutzers für Modus 1.

        eligible_users_file (Path, optional): Pfad zur JSON-Datei mit Nutzer-IDs für Modus 2.

    """

    # Stelle sicher, dass die Basis-Ausgabeverzeichnisse existieren

    results_output_dir.mkdir(parents=True, exist_ok=True)

    # user_data_dir ist ein Eingabeverzeichnis, sollte bereits existieren.

    # (user_data_dir.parent / "users").mkdir(parents=True, exist_ok=True) # Früher: (output_dir / "users")



    users_to_process = []

    all_processed_personas = {} 



    if processing_mode == 1:

        if not specific_user_id_to_process:

            print("FEHLER: PROCESSING_MODE 1 erfordert 'specific_user_id_to_process'.")

            return



        user_file_path = user_data_dir / f"{specific_user_id_to_process}.jsonl"

        

        # Optional: Dummy-Datei für Testzwecke (wenn benötigt, anpassen)

        # if not user_file_path.exists():

        #     print(f"Hinweis: Dummy-Datei {user_file_path} wird für Testzwecke erstellt.")

        #     # ... (Logik zum Erstellen von Dummy-Dateien hier, falls gewünscht) ...



        if user_file_path.exists():

            print(f"Versuche, spezifischen Nutzer zu verarbeiten: {specific_user_id_to_process}")

            if load_history_tweets_for_user(specific_user_id_to_process, user_data_dir):

                users_to_process.append(specific_user_id_to_process)

            else:

                print(f"Konnte keine History-Tweets für den spezifischen Nutzer {specific_user_id_to_process} laden.")

        else:

            print(f"Datendatei für spezifischen Nutzer {specific_user_id_to_process} nicht gefunden unter {user_file_path}")



    elif processing_mode == 2:

        if not eligible_users_file:

            print("FEHLER: PROCESSING_MODE 2 erfordert 'eligible_users_file'.")

            return

        

        # Optional: Dummy eligible_users.json Datei (wenn benötigt, anpassen)

        # if not eligible_users_file.exists():

        #     # ... (Logik zum Erstellen von Dummy-Dateien hier) ...



        if eligible_users_file.exists():

            try:

                with open(eligible_users_file, 'r', encoding='utf-8') as f:

                    loaded_user_ids = json.load(f)

                    if isinstance(loaded_user_ids, list) and all(isinstance(uid, str) for uid in loaded_user_ids):

                        users_to_process = loaded_user_ids

                        print(f"{len(users_to_process)} Nutzer-IDs aus {eligible_users_file} geladen.")

                    else:

                        print(f"FEHLER: Der Inhalt von {eligible_users_file} ist keine Liste von Strings.")

            except Exception as e:

                print(f"FEHLER beim Laden oder Verarbeiten von {eligible_users_file}: {e}")

        else:

            print(f"FEHLER: Datei {eligible_users_file} nicht gefunden.")



    elif processing_mode == 3:

        print(f"Durchsuche Verzeichnis {user_data_dir} nach allen *.jsonl Dateien als Fallback...")

        found_user_files = list(user_data_dir.glob("*.jsonl"))

        if found_user_files:

            users_to_process = [user_file.stem for user_file in found_user_files] 

            print(f"{len(users_to_process)} .jsonl Dateien gefunden, die verarbeitet werden.")

        else:

            print(f"Keine .jsonl Dateien in {user_data_dir} gefunden.")

    

    else:

        print(f"FEHLER: Unbekannter PROCESSING_MODE: {processing_mode}")

        return



    if not users_to_process:

        print("\nKeine Nutzer für die Verarbeitung ausgewählt oder gefunden. Programm wird beendet.")

    else:

        print(f"\nFolgende Nutzer werden verarbeitet ({len(users_to_process)}): {users_to_process}")



        for user_id_str_loop in users_to_process:

            print(f"\n================ BEARBEITE NUTZER: {user_id_str_loop} ================")

            

            persona_prompt_candidates = generate_persona_candidates_for_user(

                user_id_str_loop,

                user_data_dir,

                num_persona_candidates

            )



            if persona_prompt_candidates:

                all_processed_personas[user_id_str_loop] = persona_prompt_candidates

                print(f"\n--- Generierte Persona-Prompt-Kandidaten für Nutzer {user_id_str_loop} ---")

                for i, candidate in enumerate(persona_prompt_candidates):

                    print(f"Kandidat {i+1}:\n{candidate}\n")

            else:

                print(f"Keine Persona-Prompt-Kandidaten für Nutzer {user_id_str_loop} generiert oder ein Fehler ist aufgetreten.")

        

        if all_processed_personas:

            output_all_personas_file = results_output_dir / "all_generated_persona_candidates.json"

            try:

                with open(output_all_personas_file, 'w', encoding='utf-8') as f:

                    json.dump(all_processed_personas, f, indent=4, ensure_ascii=False)

                print(f"\nAlle generierten Persona-Kandidaten wurden in {output_all_personas_file} gespeichert.")

            except Exception as e:

                print(f"FEHLER beim Speichern der gesammelten Persona-Kandidaten: {e}")



    print("\nProgramm-Ausführung beendet.")

#### Imitaion

In [ ]:
# 1. Define paths and parameters

personas_input_file = DEFAULT_RESULTS_OUTPUT_DIR / "all_generated_persona_candidates.json"

imitations_output_file_v2 = DEFAULT_RESULTS_OUTPUT_DIR / "tweet_imitations_replies_and_completions.json"

user_data_dir_for_holdout = DEFAULT_JSONL_USER_DATA_DIR



NUM_REPLY_IMITATIONS_PER_PERSONA = 3

NUM_COMPLETION_IMITATIONS_PER_PERSONA = 3



# Für Tweet Completion Strategie

COMPLETION_FRAGMENT_NUM_WORDS = 7  # How many words from the start of the original tweet

MIN_WORDS_FOR_COMPLETION_ELIGIBILITY = COMPLETION_FRAGMENT_NUM_WORDS + 3 # Tweet needs to be longer



# 2. Run the imitation generation

# Make sure the persona file exists from the previous script run

if personas_input_file.exists():

    run_imitation_generation(

        personas_file=personas_input_file,

        imitations_output_file=imitations_output_file_v2,

        user_data_holdout_dir=user_data_dir_for_holdout,

        num_reply_imitations=NUM_REPLY_IMITATIONS_PER_PERSONA,

        num_completion_imitations=NUM_COMPLETION_IMITATIONS_PER_PERSONA

    )

else:

    print(f"FEHLER: Persona-Datei {personas_input_file} wurde nicht gefunden. Bitte zuerst das Persona-Generierungs-Skript ausführen.")




#### Evaluation

In [2]:
from evaluation_metrics import run_evaluation
from pathlib import Path
# Define paths relative to the project root
DEFAULT_OUTPUT_DIR = Path("Preprocessing/output_directory")
IMITATIONS_INPUT_FILE = DEFAULT_OUTPUT_DIR / "tweet_imitations_replies_and_completions.json"
EVALUATION_OUTPUT_FILE = DEFAULT_OUTPUT_DIR / "evaluation_results.json"

run_evaluation(
    imitations_input_file=IMITATIONS_INPUT_FILE,
    evaluation_output_file=EVALUATION_OUTPUT_FILE
)

Starting evaluation process...
✓ punkt already available
✓ wordnet downloaded successfully
✓ omw-1.4 downloaded successfully
✓ stopwords already available
NLTK data check completed.
ERROR: Imitations input file not found at Preprocessing\output_directory\tweet_imitations_replies_and_completions.json
